In [1]:
import db_connection as db_connect

Table alrdy exists


In [2]:
import sqlite3 as sql

In [3]:
import datetime 

In [4]:
class books:
    def __init__(self , title , author , published_year):
        self.title = title
        self.author = author
        self.published_year = published_year

    def add_to_library(self):
        with sql.connect("library_db.db") as db_connect:
            db_cursor = db_connect.cursor()
            try:
                db_cursor.execute(insert_query_table_books , (self.title , self.author , self.published_year))
            except:
                print("Your book alrdy exists")


class members:

    def __init__(self, name , join_date):
        self.name = name
        self.join_date = join_date

    def add_members_to_library(self):
        with sql.connect("library_db.db") as db_connect:
            db_cursor = db_connect.cursor()
            try:
                db_cursor.execute(insert_query_table_members , (self.name , self.join_date))
            except:
                print("This member is alrdy joined")


class admins:

    def __init__(self, username , password):
        self.username = username
        self.password = password

    def add_admin_to_library(self , username , password):
        with sql.connect("library_db.db") as db_connect:
            db_cursor = db_connect.cursor()
            try:
                db_cursor.execute(insert_query_table_admins , (self.username , self.password))
            except:
                print("This admin alrdy exists")

    def change_password(self , main_password , new_password):
        if(main_password == self.password):
            with sql.connect("library_db.db") as db_connect:
                db_cursor = db_connect.cursor()
                try:
                    db_cursor.execute(f"UPDATE admins SET password = '{new_password}' WHERE username = '{self.username}';")
                except:
                    print("Data base error try again later")
        else:
                        print("Password wrong , Your account will be logged out")

In [5]:
insert_query_table_books = """INSERT INTO Books(title , author , published_year) VALUES(?,?,?)"""
insert_query_table_members = """INSERT INTO Members(name , join_date) VALUES(?,?)"""
insert_query_table_loans = """INSERT INTO Loans(book_id , member_id , loan_date , give_back_date) VALUES(?,?,?,?)"""
insert_query_table_admins = """INSERT INTO admins(username , password) VALUES(?,?)"""

update_query_table_books = """UPDATE Books SET (?) = (?) WHERE (?) , (?)"""
update_query_table_members = """UPDATE Members SET (?) = (?) WHERE (?) , (?)"""
update_query_table_loans = """UPDATE Loans SET (?) = (?) WHERE (?) , (?)"""
update_query_table_admins = """UPDATE admins SET (?) = (?) WHERE (?) , (?)"""

In [16]:
def trim_function(string_input):
    return string_input.replace(" ","")

def making_list_from_input_string(string_input):
    string_input = string_input[0].split(",")
    for i in range(0,len(string_input)):
        string_input[i] = trim_function(string_input[i])
    return string_input

def admin_homepage(admin_permission):
    if(admin_permission == 1):
        return int(input("Enter a number for fallowing jobs:\n1 for adding books\n2 for adding member\n3 for book loaning operations\n9 for changing admin pass\n"))
    else:
        print("You dont have permission")

#-------------------------------------------------------------------------------------------------------------------



def operation_handler(operation , admin_user_object):
    if(operation == 1):
        book_info = [input("Enter your book title , author and published year and seprate them with ,:")]
        book_info = making_list_from_input_string(book_info)
        book_info = books(title=book_info[0],author=book_info[1],published_year=book_info[2])
        user_answer = input("You want to add this book to DB?(y/n)")
        if(user_answer == "y"):
            book_info.add_to_library()
            print("Your book successfully added")
            del book_info
            user_answer = input("Do you want to do any other operation(y/n):")
            if(user_answer == "y"):
                admin_homepage(1)
                del user_answer
            else:
                print("GoodBye!")

    
    if(operation == 2):
        member_name = input("Enter your member name:")
        member_info = members(name=  member_name, join_date=today_date_calculator())
        user_answer = input("You want to add this member to DB?(y/n)")
        if(user_answer == "y"):
            member_info.add_members_to_library()
            print("Your member successfully added")
            user_answer = input("Do you want to do any other operation(y/n):")
            if(user_answer == "y"):
                admin_homepage(1)
                del user_answer
                del member_info
            else:
                print("GoodBye!")
                del member_info

    if(operation == 3):
        member_id , book_id = 0 , 0
        loan_operation = int(input("Enter a number for fallowing jobs:\n1.for adding loan\n2.for ending the loan\n3.showing the loans\n"))
        if(loan_operation == 1):
            loan_info = [input("Enter the member name , the book seprate them with , :")]
            loan_info = making_list_from_input_string(loan_info)
            member_id , book_id = finding_memberid_by_name(loan_info[0] , loan_info[1])
            loan_add(member_id , book_id)
        elif(loan_operation == 2):
            member_id = input("Enter member id: ")
            book_id = input("Enter book id: ")
            with sql.connect("library_db.db") as db_connect:
                db_cursor = db_connect.cursor()
                try:
                    db_cursor.execute(f"UPDATE Loans SET Loans.give_back_date = '{today_date_calculator()}' WHERE book_id = '{book_id}' AND member_id = '{member_id}'")
                    print("Loan successfully ended")
                    admin_homepage(1)
                except:
                    print("Database Error")
                    admin_homepage(1)
        elif(loan_operation == 3):
            with sql.connect("library_db.db") as db_connect:
                db_cursor = db_connect.cursor()
                string = ""
                try:
                    db_cursor.execute("SELECT Books.title as 'book title' , Members.name as 'member name' ,loan_date as 'loan date' ,give_back_date as 'book gave back' FROM Loans JOIN Books , Members ON book_id = Books.id AND member_id = Members.id;")
                    table_result = db_cursor.fetchall()
                    print("Book Title | Member Name | Loan Date | Book Give Back Date")
                    for i in table_result:
                        for j in i:
                            string += j + "        "
                    print(string)
                except:
                    print("DataBase Error")
        
    if(operation == 9):
        main_password = input("Enter your current password:")
        new_password = input("Enter your new password:")
        new_password_check = input("Enter your new password again:")
        if(new_password == new_password_check):
            admin_user_object.change_password(main_password , new_password)
            

#-------------------------------------------------------------------------------------------------------------------
def user_and_pass_check(username , password):
    with sql.connect("library_db.db") as db_connect:
        db_cursor = db_connect.cursor()
        try:
            db_cursor.execute(f"SELECT * FROM admins WHERE admins.username = '{username}' AND admins.password = '{password}';")
            if(db_cursor.fetchone()):
                print("Login succesfull")
                return 1 
            else:
                print("Login failed Your username or password is wrong pls re run the program")
        except:
            print("Login failed Your username or password is wrong ")


def make_user_logon(username , password):
    first_admin = admins(username=username , password=password)
    first_admin.add_admin_to_library(username=first_admin.username ,password= first_admin.password)

def admin_user_object_maker(admin_permission , username , password):
    if(admin_permission == 1):   
        admin_user = admins(username=username , password= password)
        return admin_user
    
def add_admin_users(username , password):
    with sql.connect("library_db.db") as db_connect:
        db_cursor = db_connect.cursor()
        db_cursor.execute(insert_query_table_admins , (username , password) )

def add_members(name , join_date):
    with sql.connect("library_db.db") as db_connect:
        db_cursor = db_connect.cursor()
        try:
            db_cursor.execute(insert_query_table_members , (name , join_date))
        except:
            print("Your user alrdy exists")

def finding_memberid_by_name(name,title):
    with sql.connect("library_db.db") as db_connect:
        db_cursor = db_connect.cursor()
        try:
            db_cursor.execute(f"SELECT Members.id FROM Members WHERE Members.name = '{name}'")
            name = db_cursor.fetchone()
            name_output = [int(name[0])]
        except:
            print("Member doesnt exists")
        try:
            db_cursor.execute(f"SELECT Books.id FROM Books WHERE Books.title = '{title}'")
            title = db_cursor.fetchone()
            title_output = [int(title[0])]
        except:
            print("Book doesnt exists")
        try:
            return name_output[0] , title_output[0]
        except:
            print("Operation failed")
            admin_homepage()

def loan_add(member_id , book_id):
    import sqlite3 
    with sqlite3.connect("library_db.db") as db_connect:  
        db_cursor = db_connect.cursor()  
        year = db_cursor.execute(insert_query_table_loans, (book_id, member_id, today_date_calculator(), " "))  


def today_date_calculator():
    date = datetime.datetime
    today = date.today().strftime("%D")
    return today

In [18]:
operation , user_first_logon_check = 0 , 0
with sql.connect("library_db.db") as db_connect:
    db_cursor = db_connect.cursor()
    db_cursor.execute("SELECT * FROM admins")
    logon_result = db_cursor.fetchall()
    
if(logon_result == None):
    print("Make user at first Logon")
    username = input("username:")
    password = input("password:")
    make_user_logon(username , password)
    user_first_logon_check = 1

if(user_first_logon_check == 0):
    user_input = [input("Enter Your username nad password(seprate them with , ): ")]
    user_input = making_list_from_input_string(user_input)
    admin_permission = user_and_pass_check(username=user_input[0] , password=user_input[1])
    admin_user_object = admin_user_object_maker(admin_permission = admin_permission ,username = user_input[0] , password = user_input[1])
else:
    admin_permission = 1

operation = admin_homepage(admin_permission)


operation_handler(operation , admin_user_object)

KeyboardInterrupt: Interrupted by user

Book Title | Member Name | Loan Date | Book Give Back Date
Golestan        Mahdi        12/20/24                 
